In this notebook I will focus on model selection. I will test few or more models and I will choose the best to hyperaparameters tuning. So far I tested three models. Below I will present theirs brier score (smaller is better). I test all models on 10-fold time series cross validation. Except below models I tested SVM model, but it occured model took a lot of time to train. Espescially this model doesn't provide native predict_proba method and estimate it with cross validation (exact info you can find on sklearn website).

For men dataset:
1. Logistic regression: 0.20055
2. Stacking classfier: 0.20126
3. Random forest: 0.20602
4. XGboost: 0.20962
5. AdaBoost: 0.21432
6. KNN: 0.24598
7. CustomModel: 0.26291
8. MLP: 0.26731
9. Decision tree: 0.40546

For women dataset:
1. Stacking classfier: 0.19747
2. Logistic regression: 0.20135
3. Xgboost: 0.20364
4. Random forest: 0.20679
5. AdaBoost: 0.21465
6. KNN: 0.24598
7. CustomModel: 0.27187
8. MLP: 0.30917
9. Decision tree: 0.39947

## Load libraries

In [26]:
import fireducks.pandas as pd
import numpy as np
from sklearn.metrics import brier_score_loss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.base import BaseEstimator, ClassifierMixin
from custom_utils import transform_data, enrich_data
import mlflow
from xgboost import XGBClassifier

In [23]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")
mlflow.set_experiment("NCAA Classifiers")

<Experiment: artifact_location='mlflow-artifacts:/577994152132513070', creation_time=1747996421002, experiment_id='577994152132513070', last_update_time=1747996421002, lifecycle_stage='active', name='NCAA Classifiers', tags={}>

## Load and prepare data

In [10]:
wregularseason = pd.read_csv("data/WRegularSeasonCompactResults.csv")
wtourneyseason = pd.read_csv("data/WNCAATourneyCompactResults.csv")

mregularseason = pd.read_csv("data/MRegularSeasonCompactResults.csv")
mtourneyseason = pd.read_csv("data/MNCAATourneyCompactResults.csv")

mseason = pd.read_csv("data/MSeasons.csv")
wseason = pd.read_csv("data/WSeasons.csv")

wdata = pd.concat([wregularseason, wtourneyseason], axis=0)
wdata = wdata.loc[wdata.Season > 2015].reset_index(drop=True)

mdata = pd.concat([mregularseason, mtourneyseason], axis=0).reset_index(drop=True)
mdata = mdata.loc[mdata.Season > 2015].reset_index(drop=True)

wprep = transform_data(wdata)
mprep = transform_data(mdata)

mprep_enh = enrich_data(mprep, "M")
wprep_enh = enrich_data(wprep, "W")

X_featm = mprep_enh.drop(["Result", "DayNum"], axis=1)
ym = mprep_enh.Result

X_featw = wprep_enh.drop(["Result", "DayNum"], axis=1)
yw = wprep_enh.Result

tscv = TimeSeriesSplit(n_splits=10)

/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


## Model selection

### Decision tree

#### Men

In [4]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Men", "Model" : "Decision tree", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featm)):
        X_train, y_train = X_featm.iloc[train_idx], ym.iloc[train_idx]
        X_test, y_test = X_featm.iloc[test_idx], ym.iloc[test_idx]

        clf = DecisionTreeClassifier(random_state=42)
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run enchanting-koi-552 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/77666a4e3cd4460ebf4a75e19ef67948
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


#### Women

In [5]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Women", "Model" : "Decision tree", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featw)):
        X_train, y_train = X_featw.iloc[train_idx], yw.iloc[train_idx]
        X_test, y_test = X_featw.iloc[test_idx], yw.iloc[test_idx]

        clf = DecisionTreeClassifier(random_state=42)
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run amusing-mole-254 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/bb5d27525390410d9e3a24d24d3f4ec3
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


### Random forest

#### Men

In [6]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Men", "Model" : "Random forest", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featm)):
        X_train, y_train = X_featm.iloc[train_idx], ym.iloc[train_idx]
        X_test, y_test = X_featm.iloc[test_idx], ym.iloc[test_idx]

        clf = RandomForestClassifier(random_state=42)
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run tasteful-owl-526 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/20067e86297b4387866aaeb3de683548
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


#### Women

In [7]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Women", "Model" : "Random forest", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featw)):
        X_train, y_train = X_featw.iloc[train_idx], yw.iloc[train_idx]
        X_test, y_test = X_featw.iloc[test_idx], yw.iloc[test_idx]

        clf = RandomForestClassifier(random_state=42)
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run bright-ape-58 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/bfb59b92c5dd42b58671f29fef721541
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


### Logistic Regression

#### Men

In [8]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Men", "Model" : "Logistic regression", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featm)):
        X_train, y_train = X_featm.iloc[train_idx], ym.iloc[train_idx]
        X_test, y_test = X_featm.iloc[test_idx], ym.iloc[test_idx]

        clf = LogisticRegression(random_state=42)
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.

🏃 View run silent-midge-297 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/b9d4811a3bc648339f1347365d687ce0
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Women

In [9]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Women", "Model" : "Logistic regression", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featw)):
        X_train, y_train = X_featw.iloc[train_idx], yw.iloc[train_idx]
        X_test, y_test = X_featw.iloc[test_idx], yw.iloc[test_idx]

        clf = LogisticRegression(random_state=42)
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.

🏃 View run orderly-toad-890 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/e77366e813f6454782c81f78e32d9630
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### AdaBoost classifier

#### Men

In [10]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Men", "Model" : "AdaBoost", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featm)):
        X_train, y_train = X_featm.iloc[train_idx], ym.iloc[train_idx]
        X_test, y_test = X_featm.iloc[test_idx], ym.iloc[test_idx]

        clf = AdaBoostClassifier(random_state=42)
        clf.fit(X_train, y_train)
        
        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run awesome-toad-420 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/9e06be7499d74709bef3d8ae075a07cf
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


#### Women

In [11]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Women", "Model" : "AdaBoost", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featw)):
        X_train, y_train = X_featw.iloc[train_idx], yw.iloc[train_idx]
        X_test, y_test = X_featw.iloc[test_idx], yw.iloc[test_idx]

        clf = AdaBoostClassifier(random_state=42)
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run learned-stoat-382 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/bee2e9e4f06a400a89175410a9793a13
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


### KNN Classifiers

#### Men

In [13]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Men", "Model" : "KNN", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featm)):
        X_train, y_train = X_featm.iloc[train_idx], ym.iloc[train_idx]
        X_test, y_test = X_featm.iloc[test_idx], ym.iloc[test_idx]

        clf = KNeighborsClassifier()
        clf.fit(X_train, y_train)
        
        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run overjoyed-conch-607 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/02bec2d858cd4ed888c8787f3a95c858
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


#### Women

In [14]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Women", "Model" : "KNN", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featw)):
        X_train, y_train = X_featw.iloc[train_idx], yw.iloc[train_idx]
        X_test, y_test = X_featw.iloc[test_idx], yw.iloc[test_idx]

        clf = KNeighborsClassifier()
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run amazing-jay-427 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/33be9681813b410cbc0b9a049f5bba25
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


### MLP classifier

#### Men

In [15]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Men", "Model" : "MLP", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featm)):
        X_train, y_train = X_featm.iloc[train_idx], ym.iloc[train_idx]
        X_test, y_test = X_featm.iloc[test_idx], ym.iloc[test_idx]

        clf = MLPClassifier()
        clf.fit(X_train, y_train)
        
        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run skillful-ant-943 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/49a16d3361c0480fb37713f37354a39e
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


Women

In [16]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Women", "Model" : "MLP", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featw)):
        X_train, y_train = X_featw.iloc[train_idx], yw.iloc[train_idx]
        X_test, y_test = X_featw.iloc[test_idx], yw.iloc[test_idx]

        clf = MLPClassifier()
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run suave-sheep-392 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/2fb98d2aedf6491ab326d451b7a1b9f4
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


### Xgboost classifier

#### Men

In [3]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Men", "Model" : "Xgboost", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featm)):
        X_train, y_train = X_featm.iloc[train_idx], ym.iloc[train_idx]
        X_test, y_test = X_featm.iloc[test_idx], ym.iloc[test_idx]

        clf = XGBClassifier()
        clf.fit(X_train, y_train)
        
        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run fortunate-hawk-849 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/ecac6caf4127476f8cfde20ff3661507
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


#### Women

In [4]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Women", "Model" : "Xgboost", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featw)):
        X_train, y_train = X_featw.iloc[train_idx], yw.iloc[train_idx]
        X_test, y_test = X_featw.iloc[test_idx], yw.iloc[test_idx]

        clf = XGBClassifier()
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run skillful-pug-201 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/1fd2f5e78959438a97a63739bb4d9f15
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


### Custom Model (meta learner)

#### Implement model

In [21]:
class MetaModel(BaseEstimator, ClassifierMixin):
    def __init__(self):
        super().__init__()
        self.model1 = XGBClassifier()
        self.model2 = AdaBoostClassifier(random_state=42)
        self.model3 = RandomForestClassifier(random_state=42)
        self.meta_model = LogisticRegression(random_state=42)

    def fit(self, X, y):
        if X.shape[0] != y.shape[0]:
            raise ValueError(f"Unequal shapes: {X.shape[0]} != {y.shape[0]}")
        
        self.model1.fit(X, y)
        self.model2.fit(X, y)
        self.model3.fit(X, y)

        X_meta = np.concatenate([self.model1.predict_proba(X), self.model2.predict_proba(X), self.model3.predict_proba(X)], axis=1)
        self.meta_model = self.meta_model.fit(X_meta, y)
        return self
    
    def predict(self, X):
        X_meta = np.concatenate([self.model1.predict_proba(X), self.model2.predict_proba(X), self.model3.predict_proba(X)], axis=1)
        return self.meta_model.predict(X_meta)

    def predict_proba(self, X):
        X_meta = np.concatenate([self.model1.predict_proba(X), self.model2.predict_proba(X), self.model3.predict_proba(X)], axis=1)
        return self.meta_model.predict_proba(X_meta)

#### Men

In [24]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Men", "Model" : "CustomModel", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featm)):
        X_train, y_train = X_featm.iloc[train_idx], ym.iloc[train_idx]
        X_test, y_test = X_featm.iloc[test_idx], ym.iloc[test_idx]

        clf = MetaModel()
        clf.fit(X_train, y_train)
        
        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run awesome-gnu-493 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/57742eed3a5e4590bbb6a8d274d35de6
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


#### Women

In [25]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Women", "Model" : "CustomModel", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featw)):
        X_train, y_train = X_featw.iloc[train_idx], yw.iloc[train_idx]
        X_test, y_test = X_featw.iloc[test_idx], yw.iloc[test_idx]

        clf = MetaModel()
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run bemused-turtle-522 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/83e9dd65ab674f48b2d79d3f921acd0d
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


### Stacking classifier

#### Men

In [27]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Men", "Model" : "Stacking", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featm)):
        X_train, y_train = X_featm.iloc[train_idx], ym.iloc[train_idx]
        X_test, y_test = X_featm.iloc[test_idx], ym.iloc[test_idx]

        base_learners = [
            ('xgb', XGBClassifier()),
            ('ada', AdaBoostClassifier(random_state=42)),
            ('rf', RandomForestClassifier(random_state=42))
        ]

        meta_model = LogisticRegression(random_state=42)
        
        clf = StackingClassifier(
            estimators=base_learners,
            final_estimator=meta_model,
            cv=5,
            passthrough=False
        )
        clf.fit(X_train, y_train)
        
        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run placid-gnat-679 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/3ce43e84c43d42cc9a10786e6342c176
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070


#### Women

In [28]:
with mlflow.start_run():
    mlflow.set_tags({"Gender": "Women", "Model" : "Stacking", "Hyperparams" : "Basic"})
    brier_score_cv = {}

    for id_split, (train_idx, test_idx) in enumerate(tscv.split(X_featw)):
        X_train, y_train = X_featw.iloc[train_idx], yw.iloc[train_idx]
        X_test, y_test = X_featw.iloc[test_idx], yw.iloc[test_idx]

        base_learners = [
            ('xgb', XGBClassifier()),
            ('ada', AdaBoostClassifier(random_state=42)),
            ('rf', RandomForestClassifier(random_state=42))
        ]

        meta_model = LogisticRegression(random_state=42)
        
        clf = StackingClassifier(
            estimators=base_learners,
            final_estimator=meta_model,
            cv=5,
            passthrough=False
        )
        clf.fit(X_train, y_train)

        pred_proba = clf.predict_proba(X_test)[:, 1]
        brier_score_fold = brier_score_loss(y_test, pred_proba).item()
        brier_score_cv[f"Fold_{id_split}_Brier_Score"] = brier_score_fold
    brier_score_cv["Avg_Brier_Score"] = np.mean(list(brier_score_cv.values()))
    mlflow.log_metrics(brier_score_cv)

🏃 View run skillful-ox-216 at: http://127.0.0.1:8080/#/experiments/577994152132513070/runs/c3c0537a9161420c8e9c69d58f5ef4eb
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/577994152132513070
